<center> <img src ="https://i.postimg.cc/1X8H7YYt/BITS-Logo.png" width = "400" alt="BITS Pilani Logo" /> </center>

<font color='green'> <h1> <center> Dataframe Merges </center> </h1> </font>

In [1]:
import pandas as pd

pandas provides various facilities for easily combining together Series, DataFrame objects with various kinds of set logic for the indexes and relational algebra functionality in the case of join / merge-type operations.

# <b> Concat<b>

Several dataframes can be merged together using the concat function.

In [2]:
dict1 = {
            "one" : pd.Series([1, 2, 3, 4]),
            "two" : pd.Series(['a', 'b', 'c', 'd'])
}

df1 = pd.DataFrame(dict1)
df1

,one,two
0,1,a
1,2,b
2,3,c
3,4,d


In [3]:
dict2 = {
            "one" : pd.Series([5, 6, 7, 8]),
            "two" : pd.Series(['x', 'y', 'z', 'a'])
}

df2 = pd.DataFrame(dict2)
df2

,one,two
0,5,x
1,6,y
2,7,z
3,8,a


In [4]:
frames = [df1, df2]

In [5]:
pd.concat(frames)

,one,two
0,1,a
1,2,b
2,3,c
3,4,d
0,5,x
1,6,y
2,7,z
3,8,a


<b> Another Example<b>

In [6]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                    index=[0, 1, 2, 3])

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                    index=[4, 5, 6, 7])

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                    index=[8, 9, 10, 11])

frames = [df1, df2, df3]

result = pd.concat(frames)

<b> Set logic on the other axes <b>

When gluing together multiple DataFrames, you have a choice of how to handle the other axes (other than the one being concatenated). This can be done in the following three ways: <br>

- Take the union of them all, join='outer'. This is the default option as it results in zero information loss.
- Take the intersection, join='inner'.
- Use a specific index, as passed to the join_axes argument.

First, the default join='outer' behavior:

In [7]:
df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                    index=[2, 3, 6, 7])

result = pd.concat([df1, df4], axis=1, sort=False)
result

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


![image.png](attachment:image.png)

Here is the same thing with join='inner':

In [8]:
result = pd.concat([df1, df4], axis=1, join='inner')
result

,A,B,C,D,B,D,F
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


![image.png](attachment:image.png)

Lastly, suppose we just wanted to reuse the exact index from the original DataFrame:

In [10]:
df4_renamed = df4.rename(columns={'B': 'B_right', 'D': 'D_right'})
result = df1.join(df4_renamed)
result

ValueError: columns overlap but no suffix specified: Index(['B', 'D'], dtype='object')

![image.png](attachment:image.png)

In [11]:
df4_renamed = df4.rename(columns={'B': 'B_right', 'D': 'D_right'})
result = df1.join(df4_renamed, how='right')
result

ValueError: columns overlap but no suffix specified: Index(['B', 'D'], dtype='object')

# append

A useful shortcut to concat() are the append() instance methods on Series and DataFrame. These methods actually predated concat. They concatenate along axis=0, namely the index:

In [ ]:
result = df1.append(df2)
result

![image.png](attachment:image.png)

In the case of DataFrame, the indexes must be disjoint but the columns do not need to be:

In [ ]:
result = df1.append(df4, sort=False)
result

![image.png](attachment:image.png)

append may take multiple objects to concatenate:

In [ ]:
result = df1.append([df2, df3])
result

![image.png](attachment:image.png)

<b>Concatenating with mixed ndims<b>

You can concatenate a mix of Series and DataFrame objects. The Series will be transformed to DataFrame with the column name as the name of the Series.

In [ ]:
series1 = pd.Series(['X0', 'X1', 'X2', 'X3'], name='X')
result = pd.concat([df1, series1], axis=1)
result

![image.png](attachment:image.png)

If unnamed Series are passed they will be numbered consecutively.

In [ ]:
series2 = pd.Series(['_0', '_1', '_2', '_3'])
result = pd.concat([df1, series2, series2, series2], axis=1)
result

![image.png](attachment:image.png)

<b> Appending rows to a DataFrame <b>

While not especially efficient (since a new object must be created), you can append a single row to a DataFrame by passing a Series or dict to append, which returns a new DataFrame as above.

In [ ]:
s2 = pd.Series(['X0', 'X1', 'X2', 'X3'], index=['A', 'B', 'C', 'D'])
result = df1.append(s2, ignore_index=True)
result

You should use ignore_index with this method to instruct DataFrame to discard its index. You can also pass a list of dicts or Series:

In [ ]:
dicts = [{'A': 1, 'B': 2, 'C': 3, 'X': 4},
         {'A': 5, 'B': 6, 'C': 7, 'Y': 8}]

result = df1.append(dicts, ignore_index=True, sort=False)
result

![image.png](attachment:image.png)

# Database-style DataFrame merging

pandas has full-featured, high performance in-memory join operations idiomatically very similar to relational databases like SQL. These methods perform significantly better (in some cases well over an order of magnitude better) than other open source implementations (like base::merge.data.frame in R). The reason for this is careful algorithmic design and the internal layout of the data in DataFrame.

pandas provides a single function, merge(), as the entry point for all standard database join operations between DataFrame or named Series objects:

pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

In [ ]:
help(pd.merge)

The return type will be the same as left. If left is a DataFrame or named Series and right is a subclass of DataFrame, the return type will still be DataFrame.

merge is a function in the pandas namespace, and it is also available as a DataFrame instance method merge(), with the calling DataFrame being implicitly considered the left object in the join.

The related join() method, uses merge internally for the index-on-index (by default) and column(s)-on-index join. If you are joining on index only, you may wish to use DataFrame.join to save yourself some typing.

Experienced users of relational databases like SQL will be familiar with the terminology used to describe join operations between two SQL-table like structures (DataFrame objects). There are several cases to consider which are very important to understand:

- one-to-one joins: for example when joining two DataFrame objects on their indexes (which must contain unique values).
- many-to-one joins: for example when joining an index (unique) to one or more columns in a different DataFrame.
- many-to-many joins: joining columns on columns.

It is worth spending some time understanding the result of the many-to-many join case. In SQL / standard relational algebra, if a key combination appears more than once in both tables, the resulting table will have the Cartesian product of the associated data. Here is a very basic example with one unique key combination:

In [ ]:
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})

result = pd.merge(left, right, on='key')
result

![image.png](attachment:image.png)

Here is a more complicated example with multiple join keys. Only the keys appearing in left and right are present (the intersection), since how='inner' by default.

In [ ]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})

result = pd.merge(left, right, on=['key1', 'key2'])
result

![image.png](attachment:image.png)

The "how" argument to merge specifies how to determine which keys are to be included in the resulting table. If a key combination does not appear in either the left or right tables, the values in the joined table will be NA. Here is a summary of the how options and their SQL equivalent names:

![image.png](attachment:image.png)

In [ ]:
result = pd.merge(left, right, how='left', on=['key1', 'key2'])
result

![image.png](attachment:image.png)

In [ ]:
result = pd.merge(left, right, how='right', on=['key1', 'key2'])
result

![image.png](attachment:image.png)

In [ ]:
result = pd.merge(left, right, how='outer', on=['key1', 'key2'])
result

![image.png](attachment:image.png)

In [ ]:
result = pd.merge(left, right, how='inner', on=['key1', 'key2'])
result

![image.png](attachment:image.png)

#  Database-style DataFrame indexing

DataFrame.join() is a convenient method for combining the columns of two potentially differently-indexed DataFrames into a single result DataFrame. Here is a very basic example:

In [ ]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                     'B': ['B0', 'B1', 'B2']},
                    index=['K0', 'K1', 'K2'])

right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                      'D': ['D0', 'D2', 'D3']},
                     index=['K0', 'K2', 'K3'])

result = left.join(right)
result

![image.png](attachment:image.png)

In [ ]:
result = left.join(right, how='outer')
result

![image.png](attachment:image.png)

The same as above, but with how='inner'.

In [ ]:
result = left.join(right, how='inner')
result

![image.png](attachment:image.png)